In [158]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [159]:
phone_list = []

In [160]:


def extract_phone_numbers(text: str):
    # 替换掉可能干扰的中文数字和无关符号
    text = (
        text.replace("一", "1")
        .replace("－", "-")
        .replace("–", "-")
        .replace("（", "(")
        .replace("）", ")")
    )

    # 正则匹配（支持 +1、()、-、空格、斜杠、多号）
    pattern = re.compile(
        r"""
        (?:(?:\+?1[-\s]*)?)                     # 可选国家码 +1 或 1-
        (?:\(?\d{3}\)?[-\s]*)                   # 区号（含括号）
        \d{3}[-\s]*\d{4}                        # 主号段
        (?:\s*[/、,，或]+\s*(?:\(?\d{3}\)?[-\s]*\d{3}[-\s]*\d{4}))?  # 可能的第二个号码
        """,
        re.VERBOSE
    )

    matches = pattern.findall(text)

    # 清洗匹配结果
    clean_numbers = []
    for match in matches:
        number = re.sub(r"[^\d]", "", match)  # 去除非数字
        if 7 <= len(number) <= 11:  # 保留有效号码长度
            clean_numbers.append(number)
    return list(set(clean_numbers))  # 去重


In [161]:
def get_phone(info_link):
    """
        请求页面数据
    :return: 
    """
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response = requests.get(info_link, headers=headers, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    postbody = soup.find("div", class_="postbody").text.replace('\n', '').replace('\t', '')
    extract_phone_numbers_result = extract_phone_numbers(postbody)
    print(extract_phone_numbers_result,"postbody:",postbody)
    if len(extract_phone_numbers_result) == 0:
            return
    else:
        for phone_number in extract_phone_numbers_result:
            phone_list.append(phone_number)


In [162]:
def get_info_link(page):
    """
        请求页面数据
    :return: 
    """
    url = "https://c.dadi360.com/c/forums/show/53.page"
    if page != 0:
        url = f'https://c.dadi360.com/c/forums/show/{page * 90}/53.page'
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        'Pragma': 'no-cache',
        'Referer': 'https://c.dadi360.com/c/forums/show/90/53.page',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        # 'Cookie': 'JSESSIONID=23A72568409B6506BFE2169ED6119616; __utma=47376117.511507405.1761231592.1761231592.1761231592.1; __utmc=47376117; __utmz=47376117.1761231592.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); JSESSIONID=B4EBD5D72EEFD44DCF5C881BAB2FD367; __utmt=1; __utmb=47376117.34.10.1761231592',
    }
    resp = requests.get(url, headers=headers, verify=False)
    # print(resp.text)
    soup = BeautifulSoup(resp.text, 'html.parser')
    bg_small_yellows = soup.find_all("tr", class_="bg_small_yellow")
  
    for bg_small_yellow in bg_small_yellows:
        a = bg_small_yellow.find("a")
        info_link = "https://c.dadi360.com" + str(a["href"]).split(";")[0]
        get_phone(info_link)
        print(a.text, a["href"])




In [163]:
if __name__ == '__main__':

    for page in range(0,9): # 后续加页直接修改(0,max_page)
        print("page:", page)
        get_info_link(page)
    df = pd.DataFrame({"phone": phone_list})
    df.to_excel("phones.xlsx", index=False)

page: 0
  ✅法盛144-35 Sanford 統倉出售26萬9 ☎️電646-398-1788 海倫/88/ 曼哈頓快巴到中城/路易斯名高中/湯馬士哈利名高中. 皇后大學a 床$1100
['9173485670'] postbody: 皇后区长岛交界独立house 233街 87ave Queens village全新装修半图库两房一厅出租，近高速、超市，公交Q27 到法拉盛，免宠物，租金$1700全包，查信用收入。   电话9173485670
   大地360版主lushing和其他地区的房屋，请登在《中介出租》，谢谢！   大地360致力营造一个公平、健康的网络环境，再次谢谢大家的支持！同一内容、相似内容的贴子或同一联系电话，只保留一个贴子。如屋主有多间房出租，可写在贴子的主题上，内容请刊登在同一贴子里。这样在其他网友仍然可以看到您的内容的同时，也给予了他人发贴的位置。 
['9173254896'] postbody: 华盛通管 专业修理各種管道漏水，水管解冻，疏通、清洗餐館住家下水道，更换热水炉，修理水喉水管馬桶洗盆堵塞 不通不收費日夜服務 電黃先生：917-325-4896
       客人委托👏👏👏 Blvd/35Ave $3100全包顿的火车站。
['9293468797'] postbody: 境優走到貝賽高中8分鐘/走一街Bell 車站速到曼哈頓/巴士Q12/13 /31到法拉盛近銀行郵局商圈易泊車電929-346-8797
    ☎️646-712-9996 Lillian6学区 $3000
      9-569-9897厅，画展，储藏；0尺净大单间出租 招女生 租金$1000/月全包；26学区】 ，查收人和信用。
                           00全包 夏天空调另算495高速 295高速。北方大道住。$2500包水
 租有正职单身女性，单人。780全包 夏天空调50，有意可电联或者短信9296701242（下午三点之后请尽量短信，感谢步行3分钟到金城发，步行15分钟到缅街图书馆，门口超便利公交Q17/25。
['6464418356'] postbody: 45Ave156St一楼一房一厅出租，水电煤全包，附近不扫街好停车，详情请电6464418356
['9173491572'] postbody: